In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet121
from keras.models import Model
import keras.backend as K
from sklearn.metrics import roc_auc_score
import cv2
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
import pydicom
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm
%matplotlib inline

Using TensorFlow backend.


In [3]:
def get_score_and_interm_output(cv2_input_image, DenseNetImageNet121_model):

    width, height, _ = cv2_input_image.shape
    class_weights = DenseNetImageNet121_model.layers[-1].get_weights()[0]
    final_conv_layer = DenseNetImageNet121_model.layers[-3]
    get_output = K.function([DenseNetImageNet121_model.layers[0].input], 
                            [final_conv_layer.output, \
                             DenseNetImageNet121_model.layers[-1].output])
    [conv_outputs, prediction] = get_output([cv2_input_image[None,:,:,:]])
    conv_outputs = conv_outputs[0, :, :, :]
    prediction = prediction[0,:]
    
    return prediction,conv_outputs,class_weights

def get_cam_picture(prediction,conv_outputs,class_weights,dis_idx):
    #Create the class activation map.
    all_cams = []
    for m in dis_idx:
        predicted_disease = m
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[:2])
        for i, w in enumerate(class_weights[:, predicted_disease]):
                cam += w * conv_outputs[:, :, i]
        cam = cv2.resize(cam,(224,224))
        all_cams.append(cam)
    
    return all_cams

In [4]:
def cam_predict(img, model):
    img = img
    #cls_idx, true_cls = all_cls(Y_test[idx])
    preds,conv_outputs,class_weights = get_score_and_interm_output(img,model)
    dis_probs = {}
    dis_idx = []
    for idx,x in enumerate(preds):
        if(x>0.1):
            dis_idx.append(idx)
            dis_probs[cls[idx]] = x
    
    dis_idx = np.array(dis_idx)
    all_cams = get_cam_picture(preds,conv_outputs,class_weights,dis_idx)
    
    fig, ax_arr = plt.subplots(1,len(dis_idx)+1, sharex=True, sharey=True,figsize=(5,5))
    fig.set_size_inches(30,30)
    ax_arr[0].imshow(img)
    res = img
    for i in range(0,len(dis_probs)):

        ax_arr[i+1].imshow(img[:,:,0],cmap='gray')
        ax_arr[i+1].imshow(all_cams[i], alpha=0.6)
        dis_name = cls[dis_idx][i]
        ax_arr[i+1].text(0.5,0.1,"{0} : {1}".format(dis_name,np.round(dis_probs[dis_name]*100)),fontsize=14,verticalalignment='center',horizontalalignment='center', transform=ax_arr[i+1].transAxes)
        ax_arr[i+1].axis("off")
        #ax_arr[i].title(true_cls[i])
    #return res,all_cams  